<a href="https://colab.research.google.com/github/hsnam95/class2023Spring/blob/main/gpt_functions2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
import openai
import json
openai.api_key = ''

In [2]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_code(locatadd):
  import requests
  ServiceKey = 'miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D'
  URL = ('https://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList?serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D&pageNo=1&numOfRows=3&type=xml&locatadd_nm='+locatadd)
  response = requests.get(URL)
  return response

In [7]:
# Step 1: send the conversation and available functions to GPT
message = "부산광역시 부산진구 법정동코드 말해줘"
messages = [{"role": "user", "content": message}]
functions = [
    {
        "name": "get_code",
        "description": "행정구역에 대한 법정동코드를 받아옴",
        "parameters": {
            "type": "object",
            "properties": {
                "locatadd": {"type": "string", "description": "행정구역, e.g. 부산광역시 부산진구"},
            },
            "required": ["locatadd"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]

In [ ]:
if response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_code": get_code,
    }  # only one function in this example, but you can have multiple
    function_name = response_message["function_call"]["name"]
    fuction_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = fuction_to_call(
        locatadd=function_args.get("locatadd"),
    )
    function_response.text

In [ ]:
# Step 2: check if GPT wanted to call a function
if response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_order": get_order,
    }  # only one function in this example, but you can have multiple
    function_name = response_message["function_call"]["name"]
    fuction_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = fuction_to_call(
        item=function_args.get("item"),
        size=function_args.get("size"),
        number=function_args.get("number"),
        option=function_args.get("option"),
    )

    # Step 4: send the info on the function call and function response to GPT
    messages.append(response_message)  # extend conversation with assistant's reply
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
    )  # get a new response from GPT where it can see the function response
second_response